# AutoRegressive Textual Models

We'll be using RoBERTa for textual auto-regressive models to try and predict the "virality" and the "engagement" scores.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Part 1) Data Preperation



In [1]:
import pandas as pd

# Load data
train_df = pd.read_csv('train_set.csv')
validation_df = pd.read_csv('validation_set.csv')

# Normalize scores
train_df['virality_score_%'] /= 100
train_df['engagement_score_%'] /= 100

validation_df['virality_score_%'] /= 100
validation_df['engagement_score_%'] /= 100


train_df

,Unnamed: 0,video_id,virality_score_%,engagement_score_%,transcript
0,1295,iHeZcp28PTw,1.000000,0.825377,we've done red flags but what are potential g...
1,2562,VAv_qqxx5Dk,0.585549,0.660231,I had a gentleman who in a fit of schizophren...
2,2723,88uLmHImiGo,1.000000,0.740005,whoa all right good to see you how you feel...
3,3547,bU4YZyjj_OU,1.000000,0.698577,I want to see a man try and buy and that's wh...
4,759,jnWSiDNUspw,0.623538,0.836019,darling darling [Music] I said I feel it in m...
...,...,...,...,...,...
2263,330,NAdEp9fgZz4,0.551987,0.711711,how's it been being president bro we um sent ...
2264,2538,hgNp8nr24kQ,0.540674,0.681448,have you ever had a role where youve really r...
2265,433,TVWHk_CMLFc,0.615769,0.759859,God I've said I found yellow's body right out...
2266,3431,xG0fdYVdTdo,0.999990,0.703877,he got took to the cleaners by his wife the U...


In [2]:
validation_df

,Unnamed: 0,video_id,virality_score_%,engagement_score_%,transcript
0,524,HnEGqjkeH6c,1.000000,0.759935,[Music] I don't know how uh I'm shooting every...
1,1131,ePFTICqq1u8,0.679864,0.838004,my brother has fake fights in the shower and ...
2,2394,sshzL5Sc-qU,0.904137,0.732088,you got to test out things to see what you ar...
3,339,pq6-RRuv-Kg,0.672072,0.759330,I was at birthday party oh how was the birthd...
4,42,uXiyKzhVIpQ,0.632716,0.759856,what you like to do I just work I be playing ...
...,...,...,...,...,...
563,1636,ptET7rXY0BM,1.000000,0.760546,did you ever think he would be gay the moment...
564,2182,n_ht9CrAsnY,1.000000,0.592819,no answer how should I feel creatures lie here
565,1134,HGKUhttjWFQ,0.950256,0.790361,what's your darkest secret I'm ready go on I ...
566,1336,ZEJrPfn-VsA,0.508104,0.846803,if we all die today who's most likely to end ...


## Part 2) Tokenising the Inputs

We need to tokenise the inputs so RoBERTa can analyse these appropriately.

In [3]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

# Tokenization
def tokenize_data(df):
    return tokenizer(df['transcript'].tolist(), padding="max_length", truncation=True, return_tensors="pt")

train_encodings = tokenize_data(train_df)
validation_encodings = tokenize_data(validation_df)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [4]:
from transformers import RobertaModel, RobertaConfig
import torch.nn as nn

class RobertaForRegression(nn.Module):
    def __init__(self):
        super(RobertaForRegression, self).__init__()
        self.roberta = RobertaModel.from_pretrained('roberta-base')
        self.dropout = nn.Dropout(0.1)  # Adding dropout
        self.regressor = nn.Linear(self.roberta.config.hidden_size, 2)

    def forward(self, input_ids, attention_mask):
        outputs = self.roberta(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.pooler_output
        pooled_output = self.dropout(pooled_output)  # Apply dropout
        regression_output = self.regressor(pooled_output)
        return regression_output

In [5]:
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 63.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.6/281.6 kB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [6]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [7]:
import random
import string

def noise_injection(text, noise_level=0.3):
    chars = list(text)
    num_noise = int(len(chars) * noise_level)
    for _ in range(num_noise):
        idx = random.randint(0, len(chars) - 1)
        chars[idx] = random.choice(string.ascii_lowercase)  # Substitute a character
    return ''.join(chars)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm
import wandb



def train_and_validate():
    # Start a new wandb run
    torch.cuda.empty_cache()
    with wandb.init() as run:
        config = wandb.config

        # Initialize the model and move to GPU if available
        model = RobertaForRegression().to('cuda' if torch.cuda.is_available() else 'cpu')
        model.train()  # Set model to training mode

        # Define the optimizer and criterion
        optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate, weight_decay=0.01)
        criterion = nn.MSELoss()

        # Training loop
        for epoch in range(config.epochs):
            # Load and prepare data loaders
            train_loader, validation_loader = prepare_data_loaders(config.batch_size)
            model.train()
            train_loss = 0
            for inputs, masks, targets in tqdm(train_loader, desc=f"Epoch {epoch+1}/{config.epochs} - Training"):
                inputs, masks, targets = inputs.to('cuda'), masks.to('cuda'), targets.float().to('cuda')

                optimizer.zero_grad()
                outputs = model(inputs, masks)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()
                train_loss += loss.item()
                wandb.log({'training_loss_step': loss.item()})
                torch.cuda.empty_cache()

            # Calculate average training loss for the epoch
            train_loss /= len(train_loader)

            # Validation loop
            model.eval()
            validation_loss = 0
            with torch.no_grad():
                for inputs, masks, targets in tqdm(validation_loader, desc=f"Epoch {epoch+1}/{config.epochs} - Validation"):
                    inputs, masks, targets = inputs.to('cuda'), masks.to('cuda'), targets.float().to('cuda')
                    outputs = model(inputs, masks)
                    loss = criterion(outputs, targets)
                    validation_loss += loss.item()
                    wandb.log({'validation_loss_step': loss.item()})
                    torch.cuda.empty_cache()

            # Calculate average validation loss for the epoch
            validation_loss /= len(validation_loader)

            # Log metrics to wandb
            wandb.log({
                "epoch": epoch + 1,
                "train_loss": train_loss,
                "validation_loss": validation_loss
            })

        # Save the model at the end of training
        torch.save(model.state_dict(), "roberta_regression_model.pth")
        del model
        torch.cuda.empty_cache()
        wandb.save("roberta_regression_model.pth")

def augment_text(text, p_back_translate=0.3, p_noise=0.7):
    """Apply back translation and noise injection based on specified probabilities."""
    # if random.random() < p_back_translate:
    #     text = back_translate(text)  # Apply back translation
    if random.random() < p_noise:
        text = noise_injection(text)  # Apply noise injection
    return text

def prepare_data_loaders(batch_size):
    # Assuming 'train_encodings', 'train_df', 'validation_encodings', and 'validation_df' are predefined
    print("Augmenting text")
    augmented_text_df = train_df['transcript'].apply(lambda x: augment_text(x))
    print("Augmentation Complete")
    print("Encoding Starting")
    train_encodings = tokenizer(augmented_text_df.tolist(), padding="max_length", truncation=True, return_tensors="pt")
    print("Encoding Complete")

    train_dataset = TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], torch.tensor(train_df[['virality_score_%', 'engagement_score_%']].values))
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    validation_dataset = TensorDataset(validation_encodings['input_ids'], validation_encodings['attention_mask'], torch.tensor(validation_df[['virality_score_%', 'engagement_score_%']].values))
    validation_loader = DataLoader(validation_dataset, batch_size=batch_size)

    return train_loader, validation_loader


# Example wandb configuration for a sweep
sweep_config = {
    'method': 'random',  # Choose from 'grid', 'random', 'bayesian'
    'metric': {
        'name': 'validation_loss',
        'goal': 'minimize'
    },
    'parameters': {
        'learning_rate': {
            'min': 1e-7,
            'max': 1e-5
        },
        'epochs': {
            'values': [3, 5, 10]
        },
        'batch_size': {
            'values': [16, 32]
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="roberta_regression_tuning")
wandb.agent(sweep_id, train_and_validate)

Create sweep with ID: c08vlbfe
Sweep URL: https://wandb.ai/sp1d5r/roberta_regression_tuning/sweeps/c08vlbfe


wandb: Agent Starting Run: ln8zigek with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	learning_rate: 4.5249659694575685e-06
wandb: Currently logged in as: sp1d5r. Use `wandb login --relogin` to force relogin


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Augmenting text
Augmentation Complete
Encoding Starting
Encoding Complete


...

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Augmenting text
Augmentation Complete
Encoding Starting
Encoding Complete


Epoch 1/3 - Validation: 100%|██████████| 18/18 [00:03<00:00,  4.56it/s]


Augmenting text
Augmentation Complete
Encoding Starting
Encoding Complete


Epoch 2/3 - Training:  86%|████████▌ | 61/71 [00:43<00:07,  1.39it/s]

In [ ]:
wandb.finish()

In [16]:
wandb.finish()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e86e033aef0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7e86e033aef0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [19]:
torch.cuda.empty_cache()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7d863e65ba00>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7d863e65ba00>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

In [ ]:
torch.cuda.empty_cache()

Error in callback <bound method _WandbInit._resume_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7b13ed82e0b0>> (for pre_run_cell):


BrokenPipeError: [Errno 32] Broken pipe

Error in callback <bound method _WandbInit._pause_backend of <wandb.sdk.wandb_init._WandbInit object at 0x7b13ed82e0b0>> (for post_run_cell):


BrokenPipeError: [Errno 32] Broken pipe